<a href="https://colab.research.google.com/github/eTcilopp/temp_python_intro_course_gb/blob/master/a_b_testing/Homework_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа № 8<br>
На сайте запущен А/В тест с целью увеличить доход.8<br>
В приложенном excel файле вы найдете сырые данные по результатам эксперимента –8<br> user_id, тип выборки variant_name и доход принесенный пользователем revenue.8<br>
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

Решение

In [25]:

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
dataset_path = "/content/AB_Test_Results (2) (1).xlsx"
df = pd.read_excel(dataset_path)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Проведем первичный "осмотр" данных

In [4]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


Данные загрузились корректно

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


"Плохих" (null) значений нет

In [6]:
df.shape

(10000, 3)

Объем выборки приличный.

In [7]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [8]:
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

Разделим выборку на Контрольую и Тестовую.

In [9]:
control_df = df[df['VARIANT_NAME']=='control']
variant_df = df[df['VARIANT_NAME']=='variant']

Посчитаем Конверсию для обеих групп

In [46]:
control_aggregated_df = control_df.groupby('USER_ID').agg({'REVENUE': ['count', 'sum']})
# Переименование столбцов
control_aggregated_df.columns = ['USER_VISIT_COUNT', 'SUM_REVENUE']

# Добавление столбца с количеством уникальных значений прибыли для каждого ID
control_aggregated_df['ORDER_COUNT'] = control_df[control_df['REVENUE']>0].groupby('USER_ID')['REVENUE'].nunique()

variant_aggregated_df = variant_df.groupby('USER_ID').agg({'REVENUE': ['count', 'sum']})
# Переименование столбцов
variant_aggregated_df.columns = ['USER_VISIT_COUNT', 'SUM_REVENUE']

# Добавление столбца с количеством уникальных значений прибыли для каждого ID
variant_aggregated_df['ORDER_COUNT'] = variant_df[variant_df['REVENUE']>0].groupby('USER_ID')['REVENUE'].nunique()



Получим данные по заказам в каждой из групп

In [63]:
control_conversion_df = control_aggregated_df[['USER_VISIT_COUNT', 'ORDER_COUNT']].dropna()
control_conversion_df['CONVERSION'] = control_conversion_df['ORDER_COUNT']/control_conversion_df['USER_VISIT_COUNT']

variant_conversion_df = variant_aggregated_df[['USER_VISIT_COUNT', 'ORDER_COUNT']].dropna()
variant_conversion_df['CONVERSION'] = variant_conversion_df['ORDER_COUNT']/variant_conversion_df['USER_VISIT_COUNT']


Проверим распределение на нормальность.<br>
Применим тест Шапиро-Уилка

In [58]:
control_arr = control_conversion_df['CONVERSION'].values
variant_arr = variant_conversion_df['CONVERSION'].values

In [59]:
statistic_A, p_value_A = stats.shapiro(control_arr)
statistic_B, p_value_B = stats.shapiro(variant_arr)

In [60]:
print(f'p_value_A: {p_value_A:.2f}\np_value_B: {p_value_B:.2f}')

p_value_A: 0.00
p_value_B: 0.00


Значение p_value меньше 0.05, т.е. данные не распределены нормально.<br> Вероятно, это ситетически сгенерированные данные для домашней работы.


Средние значения конверсий в группах разнятся:

In [69]:
print(f"Среднее значение в контрольной группе: {control_conversion_df['CONVERSION'].mean()}")
print(f"Среднее значение в тестовой группе: {variant_conversion_df['CONVERSION'].mean()}")


Среднее значение в контрольной группе: 0.7239583333333333
Среднее значение в тестовой группе: 0.7962441314553992


Проверим различие на статистическую значимость<br>
Поскольку данные распределены не нормально, применим тест Манна-Уитни

In [70]:
statistic, p_value = stats.mannwhitneyu(control_conversion_df['CONVERSION'].values, variant_conversion_df['CONVERSION'].values)

In [72]:
print(f'p_value: {p_value}')

p_value: 0.14065699849037439


Значение p_value > уровня значимости 0.05. Следовательно, статистически значимых различий нет. Изменения не повлияли на конверсию.

Теперь посмотрим на прибыль<br>
Проверим данные на нормальность


In [75]:
statistic_A, p_value_A = stats.shapiro(control_df['REVENUE'].values)
statistic_B, p_value_B = stats.shapiro(variant_df['REVENUE'].values)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [76]:
print(f'p_value_A: {p_value_A:.2f}\np_value_B: {p_value_B:.2f}')

p_value_A: 0.00
p_value_B: 0.00


Значения p_value в обеих группах меньше 0.05 - следовательно, данные не распределены нормально

In [77]:
print(f"Среднее значение в контрольной группе: {control_df['REVENUE'].mean()}")
print(f"Среднее значение в тестовой группе: {variant_df['REVENUE'].mean()}")

Среднее значение в контрольной группе: 0.12901284109149277
Среднее значение в тестовой группе: 0.07006977671451356


Средние значения прибыли в группах разнятся.<br>
Причем, среднее значение прибыли в тестовой группе ниже!
<br>
<br>
Проверим это различие на статистическую значимость.
<br>
Поскольку данные не распределены нормально, применим тест Манна-Уитни



In [78]:
statistic, p_value = stats.mannwhitneyu(control_df['REVENUE'].values, variant_df['REVENUE'].values)
print(f'p_value: {p_value}')

p_value: 0.47825247965294926


Значение p_value больше уровня статистической значимости. Т.е. различий нет.

# Выводы:<br>
# Изменения не принесли никаких результатов. Команде продолжать работу.